# Neural Network Inference for Fraud Detection Using FHE
expected RAM usage: 6.2GB  
expected runtime: 14 seconds.

## Introduction
 
This example demonstrates a use case in the finance domain as well as demonstrating encrypted machine learning. We will demonstrate how we can use FHE along with neural networks (NN) to carry out predictions for fraud detection while keeping the data, the NN model and the prediction results encrypted at all times. The neural network and dataset determine fraudulent activities based on anonymized transactions. 

This example showcases how you are able to utilize the processing power of an untrusted environment while preserving the privacy of your sensitive data. The demonstration is split into a privileged client that has access to unencrypted data and models, and an unprivileged server that only performs homomorphic computation in a completely encrypted fashion. The data and the NN model are encrypted in a trusted client environment and then are used to carry out predictions in an untrusted or public environment. Finally, the prediction results return encrypted and can only be decrypted by the data owner in the trusted environment. The concept of providing fully outsourced, but fully encrypted computation to a cloud provider is a major motivating factor in the field of FHE. This use case example shows the capability of the SDK to build such applications.

**NOTE: while the client and server are not literally separated (nor demonstrating true remote cloud computation), the concepts generalize. One can imagine running the trusted code on local environment and the prediction code on a less trusted environment like the cloud. Additionally, we are working on FHE cloud that simplifies a lot of this.**

#### This demo uses the Credit Card Fraud Detection dataset, originally taken from: https://www.kaggle.com/mlg-ulb/creditcardfraud
This dataset contains actual anonymized transactions made by credit card holders from September 2013 and is labeled for transactions being fraudulent or genuine. See references at the bottom of the page.

## Use case

Global credit card fraud is expected to reach $35B by 2025 (Nilson Report, 2020) and since the beginning of the COVID-19 pandemic, 40% of financial services firms saw an increase in fraudulent activity (LIMRA, 2020). As well as volume effects, COVID-19 has worsened the false positive issue for over two-thirds of institutions (69%). A key challenge for many institutions is that significant changes in consumer behavior have often resulted in existing fraud detection systems wrongly identifying legitimate behavior as suspected fraud (Omdia, 2021).

With FHE, you are now able to unlock the value of regulated and sensitive PII data in the context of a less trusted cloud environment by performing AI, machine learning, and data analytic computations without ever having to decrypt. By training your AI models with additional sensitive data, you are able to achieve higher accuracy in fraud detection and reduce the false positive rate while also utilizing the many benefits of cloud computing.

FHE can also help to support a zero trust strategy and can implement strong access control measures by keeping the data, the models that process the data and the results generated encrypted and confidential; only the data owner has access to the private key and has the privilege to decrypt the results.

This demo uses SEAL backend since release 1.5.5.

<br>

## Step 1. Load the existing model and dataset into the trusted environment and encrypt them

#### 1.1. Preliminary setup

We start with a some imports. The `pyhelayers` library is python version of the HElayer library, which allows us to run AI applications under HE. `utils` is a small set of utilities used in these notebook demos (see utils.py in this folder)


In [ ]:
import pyhelayers
import utils 
from pathlib import Path
import json
import utils

utils.verify_memory()

# You can change these variables to point to your own model
# and data files.
# Also, you can see how this model was created and trained in folder data_gen 
INPUT_DIR = Path(utils.get_data_sets_dir()) / 'net_fraud'

X_H5 = INPUT_DIR / 'x_test.h5'
Y_H5 = INPUT_DIR / 'y_test.h5'
MODEL_JSON = str(INPUT_DIR / 'model.json')
MODEL_H5 = str(INPUT_DIR / 'model.h5')

batch_size=4096



#### 1.2. Load and encrypt the model

The next step loads a model that was pre-trained in the clear.

The following figure illustrates the model being used:

![Model](img/model.png)

The initialization processes involves internally an optimization step.
This step finds the best parameters for this model, and also gives us estimations on the time it would take to predict using a single core, the precision, the memory, the time it would take to encrypt/decrypt, etc. 

The input to the compilation process are some preferences that we have, specified in the `HeRunRequirements` object. In this demo we just specify that we use SEAL as the underlying backend. We also specify the batch size, how many samples we plan to provide when we perform inference.

There are many more parameters that can be specified to the optimizer. See more at the end of this notebook.


In [ ]:
he_run_req = pyhelayers.HeRunRequirements()
# Request a SEAL context
he_run_req.set_he_context_options([pyhelayers.HeContext.create(["SEAL_CKKS"])])
he_run_req.optimize_for_batch_size(batch_size)

nn = pyhelayers.NeuralNet()
nn.encode_encrypt([MODEL_JSON, MODEL_H5], he_run_req)

The above initialization processes also configured the HE encryption scheme, and generated the keys. These can be accessed via the `he_context` object.

In [ ]:
he_context = nn.get_created_he_context()

#### 1.3. Load and encrypt test data

Next, we load a few samples to test our model with.

In [ ]:
plain_samples, labels = utils.extract_batch_from_files(X_H5, Y_H5, batch_size, 0)

print('Loaded samples of shape',plain_samples.shape)

To encrypt them, we create a ModelIoEncoder for the model, responsible for encrypting the data in the correct format for our model.
The encrypted data is stored in an `EncryptedData` object.

In [ ]:
model_io_encoder = pyhelayers.ModelIoEncoder(nn)
encrypted_samples = pyhelayers.EncryptedData(he_context)
model_io_encoder.encode_encrypt(encrypted_samples, [plain_samples])


<br>

## Step 3. Perform predictions under encryption

We assume the encrypted model and data were sent over to an untrusted server (see next demos for examples how to do that).

We can now run inference with the encrypted data and encrypted NN to obtain encrypted results. This computation does not use the secret key and acts on completely encrypted values.

*NOTE: the data, the NN and the results always remain in encrypted state, even during computation.**

In [ ]:
enc_predictions = pyhelayers.EncryptedData(he_context)
with utils.elapsed_timer('predict', batch_size) as timer:
    nn.predict(enc_predictions, encrypted_samples)

<br>

## Step 4. Decrypt and assess the prediction results in the trusted environment

Assuming the results were sent back to the trusted client side, we can decrypt and decode them.

In [ ]:
plain_predictions = model_io_encoder.decrypt_decode_output(enc_predictions)
print('predictions',plain_predictions)

As this classification problem is a binary one, we will assess the results by comparing the positive and negative classifications with the true labels, also calculating the precision, recall and F1 score.

When running the model in the plain (see `data_gen/fraud_detection_demo.ipynb`), we get the following confusion matrix:  
[[4087 1]  
 [1 &emsp; 7]].  
Comparing the plain results with the confusion matrix reported below shows that the FHE model produces the same results as the plain one.

In [ ]:
accuracy=utils.assess_results(labels, plain_predictions)
if (accuracy<0.9):
    raise Exception("Accuracy too large")

## Summary

We have shown an example of running inference over encrypted data with a Neural Network. Additional notebooks in this folder demonstrate many more model types, and more ways to utilize them. Here is an overview of some of these possibilities:

* HElayers has extensive support for Neural Networks. See the online documentation for more. It supports also Logistic Regression, XGboost, and other types of models.
* Other notebooks in this folder demonstrate how to perform I/O operators on the keys and the encrypted objects to allow working with a remote untrusted server.
* The instruction `he_run_req.optimize_for_batch_size(batch_size)` in step 1.2 optimizes the system for handling the given batch size. You can try experimenting with other sizes. Generally for smaller sizes the system will have better latency and worse throughput. Other options for `he_run_req` include controlling the size of numbers being processed, the accuracy, security level, and more.
* In this demo both the Neural Network weights and the input were encrypted. For a scenario where only in the input is encrypted, set `he_run_req.set_model_encrypted(False)`, and the subsequent call to `encode_encrypt` with a call to `encode`.



<br>

References:

<sub><sup> 1.	Andrea Dal Pozzolo, Olivier Caelen, Reid A. Johnson and Gianluca Bontempi. Calibrating Probability with Undersampling for Unbalanced Classification. In Symposium on Computational Intelligence and Data Mining (CIDM), IEEE, 2015 </sup></sub>

<sub><sup> 2.	Dal Pozzolo, Andrea; Caelen, Olivier; Le Borgne, Yann-Ael; Waterschoot, Serge; Bontempi, Gianluca. Learned lessons in credit card fraud detection from a practitioner perspective, Expert systems with applications,41,10,4915-4928,2014, Pergamon </sup></sub>

<sub><sup> 3.	Dal Pozzolo, Andrea; Boracchi, Giacomo; Caelen, Olivier; Alippi, Cesare; Bontempi, Gianluca. Credit card fraud detection: a realistic modeling and a novel learning strategy, IEEE transactions on neural networks and learning systems,29,8,3784-3797,2018,IEEE </sup></sub>

<sub><sup> 4.	Dal Pozzolo, Andrea Adaptive Machine learning for credit card fraud detection ULB MLG PhD thesis (supervised by G. Bontempi) </sup></sub>

<sub><sup> 5.	Carcillo, Fabrizio; Dal Pozzolo, Andrea; Le Borgne, Yann-Aël; Caelen, Olivier; Mazzer, Yannis; Bontempi, Gianluca. Scarff: a scalable framework for streaming credit card fraud detection with Spark, Information fusion,41, 182-194,2018,Elsevier </sup></sub>

<sub><sup> 6.	Carcillo, Fabrizio; Le Borgne, Yann-Aël; Caelen, Olivier; Bontempi, Gianluca. Streaming active learning strategies for real-life credit card fraud detection: assessment and visualization, International Journal of Data Science and Analytics, 5,4,285-300,2018,Springer International Publishing </sup></sub>

<sub><sup> 7.	Bertrand Lebichot, Yann-Aël Le Borgne, Liyun He, Frederic Oblé, Gianluca Bontempi Deep-Learning Domain Adaptation Techniques for Credit Cards Fraud Detection, INNSBDDL 2019: Recent Advances in Big Data and Deep Learning, pp 78-88, 2019 </sup></sub>

<sub><sup> 8.	Fabrizio Carcillo, Yann-Aël Le Borgne, Olivier Caelen, Frederic Oblé, Gianluca Bontempi Combining Unsupervised and Supervised Learning in Credit Card Fraud Detection Information Sciences, 2019 </sup></sub>

<sub><sup> 9.	Yann-Aël Le Borgne, Gianluca Bontempi Machine Learning for Credit Card Fraud Detection - Practical Handbook </sup></sub>